In [1]:
from pathlib import Path

from dask.distributed import Client

from excited_workflow.train_models import carbon_tracker


client = Client()

In [2]:
ds_cb = Path("/data/volume_2/EXCITED_prepped_data/CT2022.flux1x1-monthly.nc")
ds_regions = Path("/data/volume_2/EXCITED_prepped_data/regions.nc")

desired_data = [
    "biomass",
    "spei",
    "modis",
    "era5_monthly",
    "era5_land_monthly",
    "copernicus_landcover"
]

x_keys = ["d2m", "mslhf", "msshf", "ssr", "str", "t2m", "spei", "NIRv", "skt",
            "stl1", "swvl1", "lccs_class"]
y_key = "bio_flux_opt"

In [3]:
ds_input = carbon_tracker.merge_datasets(desired_data, ds_cb)
ds_input

<xarray.Dataset>
Dimensions:     (time: 385, longitude: 360, latitude: 180)
Coordinates:
  * time        (time) datetime64[ns] 1990-01-01 1990-02-01 ... 2022-01-01
  * longitude   (longitude) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * latitude    (latitude) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
Data variables: (12/24)
    biomass     (time, latitude, longitude) float64 dask.array<chunksize=(385, 180, 360), meta=np.ndarray>
    spei        (time, latitude, longitude) float32 dask.array<chunksize=(96, 180, 360), meta=np.ndarray>
    NDVI        (time, latitude, longitude) float32 dask.array<chunksize=(385, 180, 360), meta=np.ndarray>
    NIRv        (time, latitude, longitude) float32 dask.array<chunksize=(385, 180, 360), meta=np.ndarray>
    d2m         (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    mslhf       (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    ...          ...
    stl4        (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    swvl1       (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    swvl2       (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    swvl3       (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    swvl4       (time, latitude, longitude) float32 dask.array<chunksize=(132, 180, 360), meta=np.ndarray>
    lccs_class  (time, latitude, longitude) float32 dask.array<chunksize=(385, 180, 360), meta=np.ndarray>
Attributes:
    title:        Changes in Global Terrestrial Live Biomass over the 21st Ce...
    version:      2
    institution:  Jet Propulsion Laboratory, California Institute of Technology
    source:       Estimates of carbon stock changes of live woody biomass fro...
    history:      \n2022-08-18: downloaded source from https://zenodo.org/rec...
    references:   \n@Xu2021{\n  author = {Xu, Liang, Saatchi, Sassan S., Yang...

In [4]:
ds_na = carbon_tracker.mask_region(ds_regions, ds_cb, ds_input)
ds_na

<xarray.Dataset>
Dimensions:           (time: 240, latitude: 45, longitude: 85)
Coordinates:
  * longitude         (longitude) float64 -139.5 -138.5 -137.5 ... -56.5 -55.5
  * latitude          (latitude) float64 15.5 16.5 17.5 18.5 ... 57.5 58.5 59.5
  * time              (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2019-12-01
Data variables: (12/26)
    bio_flux_opt      (time, latitude, longitude) float64 nan nan ... nan nan
    transcom_regions  (latitude, longitude) float64 nan nan nan ... nan nan nan
    biomass           (time, latitude, longitude) float64 nan nan ... nan nan
    spei              (time, latitude, longitude) float64 nan nan ... nan nan
    NDVI              (time, latitude, longitude) float64 nan nan ... nan nan
    NIRv              (time, latitude, longitude) float64 nan nan ... nan nan
    ...                ...
    stl4              (time, latitude, longitude) float64 nan nan ... nan nan
    swvl1             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl2             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl3             (time, latitude, longitude) float64 nan nan ... nan nan
    swvl4             (time, latitude, longitude) float64 nan nan ... nan nan
    lccs_class        (time, latitude, longitude) float32 nan nan ... nan nan
Attributes:
    averaging_period_length_hours:  744
    email:                          carbontracker.team@noaa.gov
    url:                            http://carbontracker.noaa.gov
    institution:                    NOAA Earth System Research Laboratory
    Conventions:                    CF-1.5
    history:                        Time-stamp: <Orion-login-1.HPC.MsState.Ed...
    NCO:                            netCDF Operators version 4.9.3 (Homepage ...
    version:                        CT2022 1x1 3-hourly fluxes as of 2023-01-...

In [5]:
rmse = carbon_tracker.validate_model(ds_na, 5, x_keys, y_key)

,Description,Value
0,Session id,2116
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.2,0.5,0.8,0.2,3.5,0.4


,Description,Value
0,Session id,5235
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.3,0.5,0.7,0.2,4.2,0.1


,Description,Value
0,Session id,7618
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.3,0.5,0.7,0.2,4.2,0.1


,Description,Value
0,Session id,8768
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.2,0.5,0.7,0.2,3.7,0.1


,Description,Value
0,Session id,3287
1,Target,bio_flux_opt
2,Target type,Regression
3,Original data shape,"(21325, 13)"
4,Transformed data shape,"(21325, 13)"
5,Transformed train set shape,"(14927, 13)"
6,Transformed test set shape,"(6398, 13)"
7,Numeric features,12
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.3,0.2,0.5,0.7,0.2,3.9,0.1


In [6]:
rmse

<xarray.DataArray (latitude: 41, longitude: 65)>
array([[0.61271368,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan, 0.1586413 , 0.33311229, ...,        nan,        nan,
               nan],
       [       nan, 0.14713797, 0.35610144, ...,        nan,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan, 0.60584624, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])
Coordinates:
  * latitude   (latitude) float64 37.5 38.5 39.5 40.5 ... 54.5 16.5 20.5 56.5
  * longitude  (longitude) float64 -77.5 -114.5 -104.5 ... -66.5 -65.5 -124.5